In [ ]:
# 匯入必要的模組
from dotenv import load_dotenv  # 用於載入環境變數
from langchain.prompts import ChatPromptTemplate  # 用於建立聊天提示模板
from langchain.schema.runnable import RunnableLambda, RunnableSequence  # 用於建立可執行的 Lambda 函式和序列
from langchain_ollama.llms import OllamaLLM  # 用於使用 Ollama 本地語言模型

# 載入 .env 檔案中的環境變數
load_dotenv()

# 初始化 Ollama 模型，使用 llama3.2 最新版本
model = OllamaLLM(model='llama3.2:latest')

# 建立聊天提示模板
# system 訊息：定義 AI 助手的角色和專長
# human 訊息：定義使用者的輸入格式，包含兩個變數：max_length 和 article
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一個專業的文字摘要助手，擅長將長篇文章濃縮成簡潔的重點摘要。"),
        ("human", "請將以下文章摘要成 {max_length} 字以內的重點：\n\n{article}"),
    ]
)

# 建立格式化提示的 Lambda 函式
# 接收字典參數並格式化提示模板
format_prompt = RunnableLambda(lambda x: prompt_template.format_prompt(**x))

# 建立調用模型的 Lambda 函式
# 將格式化的提示轉換為訊息並傳送給模型
invoke_model = RunnableLambda(lambda x: model.invoke(x.to_messages()))

# 建立解析輸出的 Lambda 函式
# 從模型回應中提取文字內容
parse_output = RunnableLambda(lambda x: x.content)

In [ ]:
# 定義除錯日誌函式
# 用於在 chain 執行過程中顯示每個步驟的詳細資訊
def add_debug_log(step_name):
    """
    建立一個除錯日誌函式
    
    參數:
        step_name: 步驟名稱，用於標識當前執行的階段
    
    返回:
        RunnableLambda: 可在 chain 中使用的日誌函式
    """
    def _log(x):
        # 印出分隔線
        print(f"\n{'='*60}")
        # 顯示當前步驟名稱
        print(f"[DEBUG] 步驟:{step_name}")
        # 顯示輸入資料的類型
        print(f"[DEBUG] 輸入的類型:{type(x).__name__}")
        # 如果是字串，顯示內容預覽（限制 100 字）
        if isinstance(x, str):
            print(f"[DEBUG] 內容預覽:{x[:100]}..." if len(x) > 100 else f"[DEBUG] 內容預覽:{x}")
        print(f"{'='*60}")

    return RunnableLambda(_log)

In [ ]:
# 建立處理鏈（RunnableSequence）
# 將多個步驟串連起來，依序執行文章摘要任務
chain = RunnableSequence(
    first=add_debug_log("開始處理"),  # 第一步：記錄開始處理
    middle=[
        format_prompt,  # 格式化提示模板
        add_debug_log("格式化完成"),  # 記錄格式化完成
        invoke_model,  # 調用模型生成摘要
        add_debug_log("模型調用完成"),  # 記錄模型調用完成
    ],
    last=parse_output  # 最後一步：解析輸出結果
)

# 範例文章：關於人工智慧的介紹
sample_article = """
人工智慧（AI）是電腦科學的一個分支，旨在創建能夠執行通常需要人類智能的任務的系統。
這些任務包括學習、推理、問題解決、感知和語言理解。AI 技術已經在許多領域產生重大影響，
包括醫療保健、金融、交通運輸和娛樂。機器學習是 AI 的一個子領域，它使計算機能夠從數據中學習
而無需明確編程。深度學習是機器學習的一個分支，使用人工神經網絡來模擬人腦的工作方式。
隨著技術的進步，AI 有望在未來幾年繼續改變我們的生活和工作方式。
"""

# 執行處理鏈
# 傳入文章和最大長度限制，獲得摘要結果
response = chain.invoke({
    "article": sample_article,  # 要摘要的文章
    "max_length": 100  # 摘要字數限制
})

# 顯示摘要結果
response